# Building ML Classifiers: Basic Random Forest & Gradient Boosting models

### Reading & cleaning text

In [2]:
!pip install nltk

In [3]:
import nltk
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import string
import time
import datetime
nltk.download('stopwords')

stopwords = nltk.corpus.stopwords.words('english')
ps = nltk.PorterStemmer()

data = pd.read_csv("SMSSpamCollection.tsv", sep='\t')
data.columns = ['label', 'body_text']

def count_punct(text):
    count = sum([1 for char in text if char in string.punctuation]) # Counting puntuations
    return round(count/(len(text) - text.count(" ")), 3)*100  # Percentage of punctuations

data['body_len'] = data['body_text'].apply(lambda x: len(x) - x.count(" "))
data['punct%'] = data['body_text'].apply(lambda x: count_punct(x))

def clean_text(text):
    text = "".join([word.lower() for word in text if word not in string.punctuation]) # removing puntuations and creating list
    tokens = re.split('\W+', text) # removing multiple special characters --> '\W+' ('\W' -> single special character)
    #tokens = re.findall('\w+', text)  # alternative to above statement (Split -> '\W+'; findall -> '\w+')
    text = [ps.stem(word) for word in tokens if word not in stopwords]  # stemmig the words
    return text

# TF-IDF
tfidf_vect = TfidfVectorizer(analyzer=clean_text)
X_tfidf = tfidf_vect.fit_transform(data['body_text'])
X_features = pd.concat([data['body_len'], data['punct%'], pd.DataFrame(X_tfidf.toarray())], axis=1)
print(X_features.head())

# CountVectorizer
count_vect = CountVectorizer(analyzer=clean_text)
X_count = count_vect.fit_transform(data['body_text'])
X_count_feat = pd.concat([data['body_len'], data['punct%'], pd.DataFrame(X_count.toarray())], axis=1)
print(X_count_feat.head())

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Abhi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


   body_len  punct%    0    1    2    3    4    5    6    7  ...   8094  8095  \
0       128     4.7  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...    0.0   0.0   
1        49     4.1  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...    0.0   0.0   
2        62     3.2  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...    0.0   0.0   
3        28     7.1  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...    0.0   0.0   
4       135     4.4  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...    0.0   0.0   

   8096  8097  8098  8099  8100  8101  8102  8103  
0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  
1   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  
2   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  
3   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  
4   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  

[5 rows x 8106 columns]
   body_len  punct%  0  1  2  3  4  5  6  7  ...   8094  8095  8096  8097  \
0       128     4.7  0  0  0  0  0  0  0  0  ...      0     0     0     0   
1        49     4.1  0

### Exploring RandomForestClassifier Attributes & Hyperparameters

In [4]:
from sklearn.ensemble import RandomForestClassifier

In [5]:
print(dir(RandomForestClassifier))
print(RandomForestClassifier())

['__abstractmethods__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__getstate__', '__gt__', '__hash__', '__init__', '__iter__', '__le__', '__len__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_abc_cache', '_abc_negative_cache', '_abc_negative_cache_version', '_abc_registry', '_estimator_type', '_get_param_names', '_make_estimator', '_set_oob_score', '_validate_X_predict', '_validate_estimator', '_validate_y_class_weight', 'apply', 'decision_path', 'feature_importances_', 'fit', 'get_params', 'predict', 'predict_log_proba', 'predict_proba', 'score', 'set_params']
RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=N

### Exploing RandomForestClassifier through Cross-Validation

In [6]:
from sklearn.model_selection import KFold, cross_val_score

In [7]:
rf_kfold = RandomForestClassifier(n_jobs=-1)
k_fold = KFold(n_splits=5)
cross_val_score(rf_kfold, X_features, data['label'], cv=k_fold, scoring='accuracy', n_jobs=-1)

array([0.97217235, 0.97307002, 0.97663971, 0.95956873, 0.97214735])

### Exploring RandomForestClassifier through holdout set

In [8]:
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.model_selection import train_test_split

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X_features, data['label'], test_size=0.2)

In [10]:
rf = RandomForestClassifier(n_estimators=50, max_depth=20, n_jobs=-1)
rf_model = rf.fit(X_train, y_train)

In [11]:
sorted(zip(rf_model.feature_importances_, X_train.columns), reverse=True)[0:10]

[(0.062158536016664455, 'body_len'),
 (0.03854472061430645, 3134),
 (0.035184024523319216, 1803),
 (0.029904407097169385, 4796),
 (0.023729538462981245, 2031),
 (0.020217739084740817, 7027),
 (0.01976561116667663, 7782),
 (0.018392724185500767, 7350),
 (0.01771394250538538, 5724),
 (0.015568827463156257, 7461)]

In [12]:
y_pred = rf_model.predict(X_test)
precision, recall, fscore, support = score(y_test, y_pred, pos_label='spam', average='binary')

In [13]:
print('Precision: {} / Recall: {} / Accuracy: {}'.format(round(precision, 3),
                                                        round(recall, 3),
                                                        round((y_pred==y_test).sum() / len(y_pred),3)))

Precision: 1.0 / Recall: 0.614 / Accuracy: 0.95


### Building a Grid Search

In [14]:
def train_RF(n_est, depth):
    rf1 = RandomForestClassifier(n_estimators=n_est, max_depth=depth, n_jobs=-1)
    rf_model1 = rf.fit(X_train, y_train)
    y_pred1 = rf_model.predict(X_test)
    precision1, recall1, fscore1, support1 = score(y_test, y_pred1, pos_label='spam', average='binary')
    print('Est: {} / Depth: {} ---- Precision: {} / Recall: {} / Accuracy: {}'.format(
        n_est, depth, round(precision1, 3), round(recall1, 3),
        round((y_pred1==y_test).sum() / len(y_pred1), 3)))

In [15]:
print(datetime.datetime.now())

for n_est in [10, 50, 100]:
    for depth in [10, 20, 30, None]:
        train_RF(n_est, depth)

print(datetime.datetime.now())

2019-06-05 20:24:26.606243
Est: 10 / Depth: 10 ---- Precision: 1.0 / Recall: 0.628 / Accuracy: 0.952
Est: 10 / Depth: 20 ---- Precision: 1.0 / Recall: 0.634 / Accuracy: 0.952
Est: 10 / Depth: 30 ---- Precision: 1.0 / Recall: 0.641 / Accuracy: 0.953
Est: 10 / Depth: None ---- Precision: 0.989 / Recall: 0.621 / Accuracy: 0.95
Est: 50 / Depth: 10 ---- Precision: 1.0 / Recall: 0.634 / Accuracy: 0.952
Est: 50 / Depth: 20 ---- Precision: 1.0 / Recall: 0.607 / Accuracy: 0.949
Est: 50 / Depth: 30 ---- Precision: 1.0 / Recall: 0.572 / Accuracy: 0.944
Est: 50 / Depth: None ---- Precision: 1.0 / Recall: 0.634 / Accuracy: 0.952
Est: 100 / Depth: 10 ---- Precision: 1.0 / Recall: 0.641 / Accuracy: 0.953
Est: 100 / Depth: 20 ---- Precision: 1.0 / Recall: 0.559 / Accuracy: 0.943
Est: 100 / Depth: 30 ---- Precision: 1.0 / Recall: 0.628 / Accuracy: 0.952
Est: 100 / Depth: None ---- Precision: 1.0 / Recall: 0.593 / Accuracy: 0.947
2019-06-05 20:24:52.062290


### Evaluating Random forest with GridSearchCV

In [16]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
import time

In [17]:
from sklearn.model_selection import GridSearchCV

In [18]:
rf2 = RandomForestClassifier()
param = {'n_estimators': [10, 150, 300],
        'max_depth': [30, 60, 90, None]}

gs = GridSearchCV(rf2, param, cv=5, n_jobs=-1)

In [19]:
start = time.time()
print(datetime.datetime.now())

gs_fit1 = gs.fit(X_features, data['label'])
pd.DataFrame(gs_fit1.cv_results_).sort_values('mean_test_score', ascending=False)[0:5]

2019-06-05 20:25:07.089388


C:\Users\Abhi\Anaconda3\envs\keraspy35\lib\site-packages\sklearn\utils\deprecation.py:125: FutureWarning: You are accessing a training score ('mean_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Users\Abhi\Anaconda3\envs\keraspy35\lib\site-packages\sklearn\utils\deprecation.py:125: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Users\Abhi\Anaconda3\envs\keraspy35\lib\site-packages\sklearn\utils\deprecation.py:125: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Users\Abhi\A

,mean_fit_time,mean_score_time,mean_test_score,mean_train_score,param_max_depth,param_n_estimators,params,rank_test_score,split0_test_score,split0_train_score,...,split2_test_score,split2_train_score,split3_test_score,split3_train_score,split4_test_score,split4_train_score,std_fit_time,std_score_time,std_test_score,std_train_score
7,25.937066,0.896778,0.973774,0.999147,90,150,"{'max_depth': 90, 'n_estimators': 150}",1,0.977578,0.999102,...,0.975741,0.998877,0.968553,0.999551,0.970350,0.999326,0.764614,0.098553,0.003620,0.000262
8,45.672739,1.313022,0.973235,0.999281,90,300,"{'max_depth': 90, 'n_estimators': 300}",2,0.976682,0.999326,...,0.974843,0.999102,0.968553,0.999326,0.970350,0.999102,3.700805,0.254756,0.003193,0.000168
11,47.526498,1.300575,0.972876,1.000000,None,300,"{'max_depth': None, 'n_estimators': 300}",3,0.975785,1.000000,...,0.974843,1.000000,0.968553,1.000000,0.968553,1.000000,10.577987,0.440035,0.003574,0.000000
5,39.057075,1.317984,0.972696,0.993803,60,300,"{'max_depth': 60, 'n_estimators': 300}",4,0.977578,0.994160,...,0.972147,0.994387,0.967655,0.993714,0.971249,0.993714,3.718399,0.220955,0.003355,0.000462
6,7.132938,0.611846,0.972517,0.998069,90,10,"{'max_depth': 90, 'n_estimators': 10}",5,0.973991,0.997080,...,0.974843,0.999102,0.962264,0.998877,0.973046,0.997530,0.401668,0.080657,0.005440,0.000786


In [20]:
end = time.time()
fit_time = (end - start)
print(fit_time)
print(datetime.datetime.now())

501.91995429992676
2019-06-05 20:33:29.009343


In [21]:
start = time.time()
print(datetime.datetime.now())

gs_fit2 = gs.fit(X_count_feat, data['label'])
pd.DataFrame(gs_fit2.cv_results_).sort_values('mean_test_score', ascending=False)[0:5]

2019-06-05 20:33:29.241303


C:\Users\Abhi\Anaconda3\envs\keraspy35\lib\site-packages\sklearn\utils\deprecation.py:125: FutureWarning: You are accessing a training score ('mean_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Users\Abhi\Anaconda3\envs\keraspy35\lib\site-packages\sklearn\utils\deprecation.py:125: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Users\Abhi\Anaconda3\envs\keraspy35\lib\site-packages\sklearn\utils\deprecation.py:125: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Users\Abhi\A

,mean_fit_time,mean_score_time,mean_test_score,mean_train_score,param_max_depth,param_n_estimators,params,rank_test_score,split0_test_score,split0_train_score,...,split2_test_score,split2_train_score,split3_test_score,split3_train_score,split4_test_score,split4_train_score,std_fit_time,std_score_time,std_test_score,std_train_score
11,53.319258,1.073186,0.972876,1.000000,None,300,"{'max_depth': None, 'n_estimators': 300}",1,0.978475,1.000000,...,0.973944,1.000000,0.966757,1.000000,0.969452,1.000000,11.005901,0.613986,0.004240,0.000000
8,61.110837,1.649299,0.972696,0.998832,90,300,"{'max_depth': 90, 'n_estimators': 300}",2,0.975785,0.998652,...,0.974843,0.999102,0.967655,0.999102,0.971249,0.999102,8.012952,0.285704,0.002940,0.000359
7,28.299591,0.921705,0.972517,0.998788,90,150,"{'max_depth': 90, 'n_estimators': 150}",3,0.978475,0.998877,...,0.973046,0.999326,0.966757,0.998877,0.970350,0.998204,4.562098,0.466256,0.003892,0.000365
4,27.575573,0.763009,0.971798,0.992815,60,150,"{'max_depth': 60, 'n_estimators': 150}",4,0.978475,0.993711,...,0.972147,0.993714,0.968553,0.992591,0.968553,0.992142,2.837904,0.074617,0.003637,0.000764
10,30.378347,0.988435,0.970900,1.000000,None,150,"{'max_depth': None, 'n_estimators': 150}",5,0.976682,1.000000,...,0.973046,1.000000,0.965858,1.000000,0.967655,1.000000,1.472709,0.221866,0.003851,0.000000


In [22]:
end = time.time()
fit_time = (end - start)
print(fit_time)
print(datetime.datetime.now())

593.799578666687
2019-06-05 20:43:23.041881


### Exploring GradientBoostingClassifier Attributes & Hyperparameters

In [23]:
from sklearn.ensemble import GradientBoostingClassifier

In [24]:
print(dir(GradientBoostingClassifier))
print(GradientBoostingClassifier())

['_SUPPORTED_LOSS', '__abstractmethods__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__getstate__', '__gt__', '__hash__', '__init__', '__iter__', '__le__', '__len__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_abc_cache', '_abc_negative_cache', '_abc_negative_cache_version', '_abc_registry', '_check_initialized', '_check_params', '_clear_state', '_decision_function', '_estimator_type', '_fit_stage', '_fit_stages', '_get_param_names', '_init_decision_function', '_init_state', '_is_initialized', '_make_estimator', '_resize_state', '_staged_decision_function', '_validate_estimator', '_validate_y', 'apply', 'decision_function', 'feature_importances_', 'fit', 'get_params', 'n_features', 'predict', 'predict_log_proba', 'predict_proba', 'score', 'set_params', 'staged_d

### Building a Grid Search for Gradient Boosting

In [25]:
def train_GB(est, max_depth, lr):
    gb = GradientBoostingClassifier(n_estimators=est, max_depth=max_depth, learning_rate=lr)
    gb_model = gb.fit(X_train, y_train)
    y_pred2 = gb_model.predict(X_test)
    precision2, recall2, fscore2, train_support2 = score(y_test, y_pred2, pos_label='spam', average='binary')
    print('Est: {} / Depth: {} / LR: {} ---- Precision: {} / Recall: {} / Accuracy: {}'.format(
        est, max_depth, lr, round(precision2, 3), round(recall2, 3), 
        round((y_pred2==y_test).sum()/len(y_pred2), 3)))

In [26]:
print(datetime.datetime.now())

for n_est in [50, 100, 150]:
    for max_depth in [3, 7, 11, 15]:
        for lr in [0.01, 0.1, 1]:
            train_GB(n_est, max_depth, lr)
            end = time.time()
            fit_time = (end - start)
            print(fit_time)
            print(datetime.datetime.now())
            
print(datetime.datetime.now())

2019-06-05 20:47:23.311343


C:\Users\Abhi\Anaconda3\envs\keraspy35\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


Est: 50 / Depth: 3 / LR: 0.01 ---- Precision: 0.0 / Recall: 0.0 / Accuracy: 0.87
888.8270328044891
2019-06-05 20:48:18.068335
Est: 50 / Depth: 3 / LR: 0.1 ---- Precision: 0.933 / Recall: 0.772 / Accuracy: 0.963
935.6992218494415
2019-06-05 20:49:04.940525
Est: 50 / Depth: 3 / LR: 1 ---- Precision: 0.891 / Recall: 0.786 / Accuracy: 0.96
972.6200978755951
2019-06-05 20:49:41.861401
Est: 50 / Depth: 7 / LR: 0.01 ---- Precision: 1.0 / Recall: 0.007 / Accuracy: 0.871
1086.8177649974823
2019-06-05 20:51:36.059068
Est: 50 / Depth: 7 / LR: 0.1 ---- Precision: 0.917 / Recall: 0.841 / Accuracy: 0.969
1191.3979365825653
2019-06-05 20:53:20.639239
Est: 50 / Depth: 7 / LR: 1 ---- Precision: 0.881 / Recall: 0.814 / Accuracy: 0.961
1257.8169362545013
2019-06-05 20:54:27.058239
Est: 50 / Depth: 11 / LR: 0.01 ---- Precision: 1.0 / Recall: 0.021 / Accuracy: 0.873
1431.3586540222168
2019-06-05 20:57:20.599957
Est: 50 / Depth: 11 / LR: 0.1 ---- Precision: 0.89 / Recall: 0.834 / Accuracy: 0.965
1593.454758

### Evaluating Gradient Boosting with GridSearchCV

In [27]:
#from sklearn.model_selection import GridSearchCV

gb2 = GradientBoostingClassifier()
param = {
    'n_estimators': [100, 150], 
    'max_depth': [7, 11, 15],
    'learning_rate': [0.1]
}

clf = GridSearchCV(gb2, param, cv=5, n_jobs=-1)

In [ ]:
start = time.time()
print(datetime.datetime.now())

cv_fit = clf.fit(X_features, data['label'])
pd.DataFrame(cv_fit.cv_results_).sort_values('mean_test_score', ascending=False)[0:5]

2019-06-05 22:19:54.752257


In [ ]:
end = time.time()
fit_time = (end - start)
print(fit_time)
print(datetime.datetime.now())

In [ ]:
start = time.time()
print(datetime.datetime.now())

cv_fit = clf.fit(X_count_feat, data['label'])
pd.DataFrame(cv_fit.cv_results_).sort_values('mean_test_score', ascending=False)[0:5]

In [ ]:
end = time.time()
fit_time = (end - start)
print(fit_time)
print(datetime.datetime.now())